In [ ]:
import pandas as pd
from ics import Calendar, Event, DisplayAlarm
from datetime import datetime, timedelta

# read scraped output
df = pd.read_csv("output.csv", names=["artist","url","artist_country","location","date_string","insta","spotify","rf_short_description"])
df = df.set_index('artist')

# read ratings and comments from google sheet
SHEET_ID = '1uu3_S_e2zR5O2cTbknIUh4bG1GymBtdwieFwZynTPRY'
SHEET_NAME = 'transform'
url = f'https://docs.google.com/spreadsheets/d/{SHEET_ID}/gviz/tq?tqx=out:csv&sheet={SHEET_NAME}'
df_ratings = pd.read_csv(url,usecols=['🧑‍🎤 artist 👩‍🎤','⭐ Mikael rating','⭐ Line rating','💭 comment','emoji-beskrivelse 🎨'])

# rename
df_ratings = df_ratings.rename(columns={'🧑‍🎤 artist 👩‍🎤':'artist','⭐ Mikael rating':'m_rating','⭐ Line rating':'l_rating','💭 comment':'comment','emoji-beskrivelse 🎨':'emoji_beskrivelse'})
df_ratings = df_ratings.set_index('artist')



In [ ]:
# join website info with ratings  
df = df.join(df_ratings, how='outer', lsuffix='_left', rsuffix='_right')

# replace nulls w blanks to stop calendar export from complaining
df[['location','comment','m_rating','l_rating','emoji_beskrivelse']] = df[['location','comment','m_rating','l_rating','emoji_beskrivelse']].fillna(' ')


In [ ]:
# get starttime, fill nulls with timestamp 13:37. Consider changing it to whole day event. But might not be relevant
df['start_time'] = df['date_string'].str.extract(r'(\d{2}:\d{2})').fillna('13:37')


# strip irrelevant locale information; could probably be made redundant
df['new_date_string'] = df['date_string'].replace(to_replace=r'.*dag (\d{1,2})\. (\w{3})\w', value=r'\1\2', regex=True)

# convert to timestamp w timezone; ics package needs utc input
df['start_datetime'] = pd.to_datetime(df['new_date_string'] + df['start_time'], format='%d%b %Y%H:%M').dt.tz_localize('CET')

# events after midnight are "placed" on the prior days programme. Below function shifts this one day to the correct calendar date
df['start_datetime'] = df['start_datetime'].apply(lambda x: x + pd.Timedelta(days=1) if x.hour < 8 else x)

## todo handle time of day for ical. Example below
df[['date_string','start_datetime','start_time']][0:].head()

In [ ]:
#df['location'].unique()
stages = {
    'Apollo': '🚀 Apollo',
    'Platform': 'Platform',
    'Festival Site': '🏟️ Festival Site',
    'Eos': '👩 Eos',
    'Avalon': '🏝️ Avalon',
    'Campsite': '⛺ Campsite',
    'Gaia': '🌍 Gaia',
    'Arena': '🎪 Arena',
    'Destroy Me Once, Destroy Me Twice': '☠️ Destroy Me Once, Destroy Me Twice',
    'Gloria': '🏝️ Gloria',
    'Flokkr': '📣 Flokkr',
    'Orange': '🟠 Orange',
    ' ': '❔',
    'Food Is Now': '🥫 Food Is Now'}

df['location'] = df['location'].map(stages)

In [ ]:
# write most important concerts to ical
c = Calendar()
for index, row in df[(df['m_rating'] == '🤯JA!') | (df['m_rating'] == '👍Ja' )].iterrows():
    e = Event()

    abbreviated_comment = ' | ' + row['comment'] if len(row['comment']) > 1 else ''
    line_rating = ' | L: ' + row['l_rating'][0] if len(row['l_rating']) > 1 else ''

    # add name and location + short info 
    e.name = row['m_rating'][0] + " " + index + line_rating
    e.location = row['location'] + abbreviated_comment

    e.begin = row['start_datetime']

    # assume all concerts are 1 hour
    e.end = row['start_datetime'] + timedelta(hours=1) 

    # add url and description
    e.url = row['url']
    e.description = '"' + row['rf_short_description'] + '"\n▶️ ' + row['spotify'] + '\n 📸 ' + row['insta']

    # only add alarm for the most important concerts
    if row['m_rating'] == '🤯JA!':
        a=DisplayAlarm()
        a.trigger=timedelta(hours=-1)
        e.alarms=[a]
    
    c.events.add(e)

# c.events
with open('most_important.ics', 'w') as my_file:
    my_file.writelines(c.serialize_iter())


In [ ]:
# thx for the random_emoji_string-function, chatGPT <3

import random

emojis = [ # from https://getemoji.com/
    '😀','😃','😄','😁','😆','😅','😂','🤣','🥲','🥹','☺️','😊','😇','🙂','🙃','😉','😌','😍','🥰','😘','😗','😙','😚','😋','😛','😝','😜','🤪','🤨','🧐','🤓','😎','🥸','🤩','🥳','😏','😒','😞','😔','😟','😕','🙁','☹️','😣','😖','😫','😩','🥺','😢','😭','😮‍💨','😤','😠','😡','🤬','🤯','😳','🥵','🥶','😱','😨','😰','😥','😓','🫣','🤗','🫡','🤔','🫢','🤭','🤫','🤥','😶','😶‍🌫️','😐','😑','😬','🫨','🫠','🙄','😯','😦','😧','😮','😲','🥱','😴','🤤','😪','😵','😵‍💫','🫥','🤐','🥴','🤢','🤮','🤧','😷','🤒','🤕','🤑','🤠','😈','👿','👹','👺','🤡','💩','👻','💀','☠️','👽','👾','🤖','🎃','😺','😸','😹','😻','😼','😽','🙀','😿','😾'
    ,'👋','🤚','🖐','✋','🖖','👌','🤌','🤏','✌️','🤞','🫰','🤟','🤘','🤙','🫵','🫱','🫲','🫸','🫷','🫳','🫴','👈','👉','👆','🖕','👇','☝️','👍','👎','✊','👊','🤛','🤜','👏','🫶','🙌','👐','🤲','🤝','🙏','✍️','💅','🤳','💪','🦾','🦵','🦿','🦶','👣','👂','🦻','👃','🫀','🫁','🧠','🦷','🦴','👀','👁','👅','👄','🫦','💋','🩸'
    ,'👶','👧','🧒','👦','👩','🧑','👨','👩‍🦱','🧑‍🦱','👨‍🦱','👩‍🦰','🧑‍🦰','👨‍🦰','👱‍♀️','👱','👱‍♂️','👩‍🦳','🧑‍🦳','👨‍🦳','👩‍🦲','🧑‍🦲','👨‍🦲','🧔‍♀️','🧔','🧔‍♂️','👵','🧓','👴','👲','👳‍♀️','👳','👳‍♂️','🧕','👮‍♀️','👮','👮‍♂️','👷‍♀️','👷','👷‍♂️','💂‍♀️','💂','💂‍♂️','🕵️‍♀️','🕵️','🕵️‍♂️','👩‍⚕️','🧑‍⚕️','👨‍⚕️','👩‍🌾','🧑‍🌾','👨‍🌾','👩‍🍳','🧑‍🍳','👨‍🍳','👩‍🎓','🧑‍🎓','👨‍🎓','👩‍🎤','🧑‍🎤','👨‍🎤','👩‍🏫','🧑‍🏫','👨‍🏫','👩‍🏭','🧑‍🏭','👨‍🏭','👩‍💻','🧑‍💻','👨‍💻','👩‍💼','🧑‍💼','👨‍💼','👩‍🔧','🧑‍🔧','👨‍🔧','👩‍🔬','🧑‍🔬','👨‍🔬','👩‍🎨','🧑‍🎨','👨‍🎨','👩‍🚒','🧑‍🚒','👨‍🚒','👩‍✈️','🧑‍✈️','👨‍✈️','👩‍🚀','🧑‍🚀','👨‍🚀','👩‍⚖️','🧑‍⚖️','👨‍⚖️','👰‍♀️','👰','👰‍♂️','🤵‍♀️','🤵','🤵‍♂️','👸','🫅','🤴','🥷','🦸‍♀️','🦸','🦸‍♂️','🦹‍♀️','🦹','🦹‍♂️','🤶','🧑‍🎄','🎅','🧙‍♀️','🧙','🧙‍♂️','🧝‍♀️','🧝','🧝‍♂️','🧛‍♀️','🧛','🧛‍♂️','🧟‍♀️','🧟','🧟‍♂️','🧞‍♀️','🧞','🧞‍♂️','🧜‍♀️','🧜','🧜‍♂️','🧚‍♀️','🧚','🧚‍♂️','🧌','👼','🤰','🫄','🫃','🤱','👩‍🍼','🧑‍🍼','👨‍🍼','🙇‍♀️','🙇','🙇‍♂️','💁‍♀️','💁','💁‍♂️','🙅‍♀️','🙅','🙅‍♂️','🙆‍♀️','🙆','🙆‍♂️','🙋‍♀️','🙋','🙋‍♂️','🧏‍♀️','🧏','🧏‍♂️','🤦‍♀️','🤦','🤦‍♂️','🤷‍♀️','🤷','🤷‍♂️','🙎‍♀️','🙎','🙎‍♂️','🙍‍♀️','🙍','🙍‍♂️','💇‍♀️','💇','💇‍♂️','💆‍♀️','💆','💆‍♂️','🧖‍♀️','🧖','🧖‍♂️','💅','🤳','💃','🕺','👯‍♀️','👯','👯‍♂️','🕴','👩‍🦽','🧑‍🦽','👨‍🦽','👩‍🦼','🧑‍🦼','👨‍🦼','🚶‍♀️','🚶','🚶‍♂️','👩‍🦯','🧑‍🦯','👨‍🦯','🧎‍♀️','🧎','🧎‍♂️','🏃‍♀️','🏃','🏃‍♂️','🧍‍♀️','🧍','🧍‍♂️','👭','🧑‍🤝‍🧑','👬','👫','👩‍❤️‍👩','💑','👨‍❤️‍👨','👩‍❤️‍👨','👩‍❤️‍💋‍👩','💏','👨‍❤️‍💋‍👨','👩‍❤️‍💋‍👨','👪','👨‍👩‍👦','👨‍👩‍👧','👨‍👩‍👧‍👦','👨‍👩‍👦‍👦','👨‍👩‍👧‍👧','👨‍👨‍👦','👨‍👨‍👧','👨‍👨‍👧‍👦','👨‍👨‍👦‍👦','👨‍👨‍👧‍👧','👩‍👩‍👦','👩‍👩‍👧','👩‍👩‍👧‍👦','👩‍👩‍👦‍👦','👩‍👩‍👧‍👧','👨‍👦','👨‍👦‍👦','👨‍👧','👨‍👧‍👦','👨‍👧‍👧','👩‍👦','👩‍👦‍👦','👩‍👧','👩‍👧‍👦','👩‍👧‍👧','🗣','👤','👥','🫂'
    ,'🧳','🌂','☂️','🧵','🪡','🪢','🪭','🧶','👓','🕶','🥽','🥼','🦺','👔','👕','👖','🧣','🧤','🧥','🧦','👗','👘','🥻','🩴','🩱','🩲','🩳','👙','👚','👛','👜','👝','🎒','👞','👟','🥾','🥿','👠','👡','🩰','👢','👑','👒','🎩','🎓','🧢','⛑','🪖','💄','💍','💼'
    ,'🐶','🐱','🐭','🐹','🐰','🦊','🐻','🐼','🐻‍❄️','🐨','🐯','🦁','🐮','🐷','🐽','🐸','🐵','🙈','🙉','🙊','🐒','🐔','🐧','🐦','🐦‍⬛','🐤','🐣','🐥','🦆','🦅','🦉','🦇','🐺','🐗','🐴','🦄','🐝','🪱','🐛','🦋','🐌','🐞','🐜','🪰','🪲','🪳','🦟','🦗','🕷','🕸','🦂','🐢','🐍','🦎','🦖','🦕','🐙','🦑','🦐','🦞','🦀','🪼','🪸','🐡','🐠','🐟','🐬','🐳','🐋','🦈','🐊','🐅','🐆','🦓','🫏','🦍','🦧','🦣','🐘','🦛','🦏','🐪','🐫','🦒','🦘','🦬','🐃','🐂','🐄','🐎','🐖','🐏','🐑','🦙','🐐','🦌','🫎','🐕','🐩','🦮','🐕‍🦺','🐈','🐈‍⬛','🪽','🪶','🐓','🦃','🦤','🦚','🦜','🦢','🪿','🦩','🕊','🐇','🦝','🦨','🦡','🦫','🦦','🦥','🐁','🐀','🐿','🦔','🐾','🐉','🐲','🌵','🎄','🌲','🌳','🌴','🪹','🪺','🪵','🌱','🌿','☘️','🍀','🎍','🪴','🎋','🍃','🍂','🍁','🍄','🐚','🪨','🌾','💐','🌷','🪷','🌹','🥀','🌺','🌸','🪻','🌼','🌻','🌞','🌝','🌛','🌜','🌚','🌕','🌖','🌗','🌘','🌑','🌒','🌓','🌔','🌙','🌎','🌍','🌏','🪐','💫','⭐️','🌟','✨','⚡️','☄️','💥','🔥','🌪','🌈','☀️','🌤','⛅️','🌥','☁️','🌦','🌧','⛈','🌩','🌨','❄️','☃️','⛄️','🌬','💨','💧','💦','🫧','☔️','☂️','🌊','🌫'
    ,'🍏','🍎','🍐','🍊','🍋','🍌','🍉','🍇','🍓','🫐','🍈','🍒','🍑','🥭','🍍','🥥','🥝','🍅','🍆','🥑','🥦','🫛','🥬','🥒','🌶','🫑','🌽','🥕','🫒','🧄','🧅','🫚','🥔','🍠','🫘','🥐','🥯','🍞','🥖','🥨','🧀','🥚','🍳','🧈','🥞','🧇','🥓','🥩','🍗','🍖','🦴','🌭','🍔','🍟','🍕','🫓','🥪','🥙','🧆','🌮','🌯','🫔','🥗','🥘','🫕','🥫','🍝','🍜','🍲','🍛','🍣','🍱','🥟','🦪','🍤','🍙','🍚','🍘','🍥','🥠','🥮','🍢','🍡','🍧','🍨','🍦','🥧','🧁','🍰','🎂','🍮','🍭','🍬','🍫','🍿','🍩','🍪','🌰','🥜','🍯','🥛','🍼','🫖','☕️','🍵','🧃','🥤','🧋','🫙','🍶','🍺','🍻','🥂','🍷','🫗','🥃','🍸','🍹','🧉','🍾','🧊','🥄','🍴','🍽','🥣','🥡','🥢','🧂'
    ,'⚽️','🏀','🏈','⚾️','🥎','🎾','🏐','🏉','🥏','🎱','🪀','🏓','🏸','🏒','🏑','🥍','🏏','🪃','🥅','⛳️','🪁','🏹','🎣','🤿','🥊','🥋','🎽','🛹','🛼','🛷','⛸','🥌','🎿','⛷','🏂','🪂','🏋️‍♀️','🏋️','🏋️‍♂️','🤼‍♀️','🤼','🤼‍♂️','🤸‍♀️','🤸','🤸‍♂️','⛹️‍♀️','⛹️','⛹️‍♂️','🤺','🤾‍♀️','🤾','🤾‍♂️','🏌️‍♀️','🏌️','🏌️‍♂️','🏇','🧘‍♀️','🧘','🧘‍♂️','🏄‍♀️','🏄','🏄‍♂️','🏊‍♀️','🏊','🏊‍♂️','🤽‍♀️','🤽','🤽‍♂️','🚣‍♀️','🚣','🚣‍♂️','🧗‍♀️','🧗','🧗‍♂️','🚵‍♀️','🚵','🚵‍♂️','🚴‍♀️','🚴','🚴‍♂️','🏆','🥇','🥈','🥉','🏅','🎖','🏵','🎗','🎫','🎟','🎪','🤹','🤹‍♂️','🤹‍♀️','🎭','🩰','🎨','🎬','🎤','🎧','🎼','🎹','🥁','🪘','🪇','🎷','🎺','🪗','🎸','🪕','🎻','🪈','🎲','♟','🎯','🎳','🎮','🎰','🧩'
    ,'🚗','🚕','🚙','🚌','🚎','🏎','🚓','🚑','🚒','🚐','🛻','🚚','🚛','🚜','🦯','🦽','🦼','🛴','🚲','🛵','🏍','🛺','🚨','🚔','🚍','🚘','🚖','🛞','🚡','🚠','🚟','🚃','🚋','🚞','🚝','🚄','🚅','🚈','🚂','🚆','🚇','🚊','🚉','✈️','🛫','🛬','🛩','💺','🛰','🚀','🛸','🚁','🛶','⛵️','🚤','🛥','🛳','⛴','🚢','⚓️','🛟','🪝','⛽️','🚧','🚦','🚥','🚏','🗺','🗿','🗽','🗼','🏰','🏯','🏟','🎡','🎢','🛝','🎠','⛲️','⛱','🏖','🏝','🏜','🌋','⛰','🏔','🗻','🏕','⛺️','🛖','🏠','🏡','🏘','🏚','🏗','🏭','🏢','🏬','🏣','🏤','🏥','🏦','🏨','🏪','🏫','🏩','💒','🏛','⛪️','🕌','🕍','🛕','🕋','⛩','🛤','🛣','🗾','🎑','🏞','🌅','🌄','🌠','🎇','🎆','🌇','🌆','🏙','🌃','🌌','🌉','🌁'
    ,'⌚️','📱','📲','💻','⌨️','🖥','🖨','🖱','🖲','🕹','🗜','💽','💾','💿','📀','📼','📷','📸','📹','🎥','📽','🎞','📞','☎️','📟','📠','📺','📻','🎙','🎚','🎛','🧭','⏱','⏲','⏰','🕰','⌛️','⏳','📡','🔋','🪫','🔌','💡','🔦','🕯','🪔','🧯','🛢','🛍️','💸','💵','💴','💶','💷','🪙','💰','💳','💎','⚖️','🪮','🪜','🧰','🪛','🔧','🔨','⚒','🛠','⛏','🪚','🔩','⚙️','🪤','🧱','⛓','🧲','🔫','💣','🧨','🪓','🔪','🗡','⚔️','🛡','🚬','⚰️','🪦','⚱️','🏺','🔮','📿','🧿','🪬','💈','⚗️','🔭','🔬','🕳','🩹','🩺','🩻','🩼','💊','💉','🩸','🧬','🦠','🧫','🧪','🌡','🧹','🪠','🧺','🧻','🚽','🚰','🚿','🛁','🛀','🧼','🪥','🪒','🧽','🪣','🧴','🛎','🔑','🗝','🚪','🪑','🛋','🛏','🛌','🧸','🪆','🖼','🪞','🪟','🛍','🛒','🎁','🎈','🎏','🎀','🪄','🪅','🎊','🎉','🪩','🎎','🏮','🎐','🧧','✉️','📩','📨','📧','💌','📥','📤','📦','🏷','🪧','📪','📫','📬','📭','📮','📯','📜','📃','📄','📑','🧾','📊','📈','📉','🗒','🗓','📆','📅','🗑','🪪','📇','🗃','🗳','🗄','📋','📁','📂','🗂','🗞','📰','📓','📔','📒','📕','📗','📘','📙','📚','📖','🔖','🧷','🔗','📎','🖇','📐','📏','🧮','📌','📍','✂️','🖊','🖋','✒️','🖌','🖍','📝','✏️','🔍','🔎','🔏','🔐','🔒','🔓'
    ,'❤️','🩷','🧡','💛','💚','💙','🩵','💜','🖤','🩶','🤍','🤎','❤️‍🔥','❤️‍🩹','💔','❣️','💕','💞','💓','💗','💖','💘','💝','💟','☮️','✝️','☪️','🪯','🕉','☸️','✡️','🔯','🕎','☯️','☦️','🛐','⛎','♈️','♉️','♊️','♋️','♌️','♍️','♎️','♏️','♐️','♑️','♒️','♓️','🆔','⚛️','🉑','☢️','☣️','📴','📳','🈶','🈚️','🈸','🈺','🈷️','✴️','🆚','💮','🉐','㊙️','㊗️','🈴','🈵','🈹','🈲','🅰️','🅱️','🆎','🆑','🅾️','🆘','❌','⭕️','🛑','⛔️','📛','🚫','💯','💢','♨️','🚷','🚯','🚳','🚱','🔞','📵','🚭','❗️','❕','❓','❔','‼️','⁉️','🔅','🔆','〽️','⚠️','🚸','🔱','⚜️','🔰','♻️','✅','🈯️','💹','❇️','✳️','❎','🌐','💠','Ⓜ️','🌀','💤','🏧','🚾','♿️','🅿️','🛗','🈳','🈂️','🛂','🛃','🛄','🛅','🚹','🚺','🚼','⚧','🚻','🚮','🎦','🛜','📶','🈁','🔣','ℹ️','🔤','🔡','🔠','🆖','🆗','🆙','🆒','🆕','🆓','0️⃣','1️⃣','2️⃣','3️⃣','4️⃣','5️⃣','6️⃣','7️⃣','8️⃣','9️⃣','🔟','🔢','#️⃣','*️⃣','⏏️','▶️','⏸','⏯','⏹','⏺','⏭','⏮','⏩','⏪','⏫','⏬','◀️','🔼','🔽','➡️','⬅️','⬆️','⬇️','↗️','↘️','↙️','↖️','↕️','↔️','↪️','↩️','⤴️','⤵️','🔀','🔁','🔂','🔄','🔃','🎵','🎶','➕','➖','➗','✖️','🟰','♾','💲','💱','™️','©️','®️','〰️','➰','➿','🔚','🔙','🔛','🔝','🔜','✔️','☑️','🔘','🔴','🟠','🟡','🟢','🔵','🟣','⚫️','⚪️','🟤','🔺','🔻','🔸','🔹','🔶','🔷','🔳','🔲','▪️','▫️','◾️','◽️','◼️','◻️','🟥','🟧','🟨','🟩','🟦','🟪','⬛️','⬜️','🟫','🔈','🔇','🔉','🔊','🔔','🔕','📣','📢','👁‍🗨','💬','💭','🗯','♠️','♣️','♥️','♦️','🃏','🎴','🀄️','🕐','🕑','🕒','🕓','🕔','🕕','🕖','🕗','🕘','🕙','🕚','🕛','🕜','🕝','🕞','🕟','🕠','🕡','🕢','🕣','🕤','🕥','🕦','🕧'
    ,'🏳️','🏴','🏁','🚩','🏳️‍🌈','🏳️‍⚧️','🏴‍☠️','🇦🇫','🇦🇽','🇦🇱','🇩🇿','🇦🇸','🇦🇩','🇦🇴','🇦🇮','🇦🇶','🇦🇬','🇦🇷','🇦🇲','🇦🇼','🇦🇺','🇦🇹','🇦🇿','🇧🇸','🇧🇭','🇧🇩','🇧🇧','🇧🇾','🇧🇪','🇧🇿','🇧🇯','🇧🇲','🇧🇹','🇧🇴','🇧🇦','🇧🇼','🇧🇷','🇮🇴','🇻🇬','🇧🇳','🇧🇬','🇧🇫','🇧🇮','🇰🇭','🇨🇲','🇨🇦','🇮🇨','🇨🇻','🇧🇶','🇰🇾','🇨🇫','🇹🇩','🇨🇱','🇨🇳','🇨🇽','🇨🇨','🇨🇴','🇰🇲','🇨🇬','🇨🇩','🇨🇰','🇨🇷','🇨🇮','🇭🇷','🇨🇺','🇨🇼','🇨🇾','🇨🇿','🇩🇰','🇩🇯','🇩🇲','🇩🇴','🇪🇨','🇪🇬','🇸🇻','🇬🇶','🇪🇷','🇪🇪','🇪🇹','🇪🇺','🇫🇰','🇫🇴','🇫🇯','🇫🇮','🇫🇷','🇬🇫','🇵🇫','🇹🇫','🇬🇦','🇬🇲','🇬🇪','🇩🇪','🇬🇭','🇬🇮','🇬🇷','🇬🇱','🇬🇩','🇬🇵','🇬🇺','🇬🇹','🇬🇬','🇬🇳','🇬🇼','🇬🇾','🇭🇹','🇭🇳','🇭🇰','🇭🇺','🇮🇸','🇮🇳','🇮🇩','🇮🇷','🇮🇶','🇮🇪','🇮🇲','🇮🇱','🇮🇹','🇯🇲','🇯🇵','🎌','🇯🇪','🇯🇴','🇰🇿','🇰🇪','🇰🇮','🇽🇰','🇰🇼','🇰🇬','🇱🇦','🇱🇻','🇱🇧','🇱🇸','🇱🇷','🇱🇾','🇱🇮','🇱🇹','🇱🇺','🇲🇴','🇲🇰','🇲🇬','🇲🇼','🇲🇾','🇲🇻','🇲🇱','🇲🇹','🇲🇭','🇲🇶','🇲🇷','🇲🇺','🇾🇹','🇲🇽','🇫🇲','🇲🇩','🇲🇨','🇲🇳','🇲🇪','🇲🇸','🇲🇦','🇲🇿','🇲🇲','🇳🇦','🇳🇷','🇳🇵','🇳🇱','🇳🇨','🇳🇿','🇳🇮','🇳🇪','🇳🇬','🇳🇺','🇳🇫','🇰🇵','🇲🇵','🇳🇴','🇴🇲','🇵🇰','🇵🇼','🇵🇸','🇵🇦','🇵🇬','🇵🇾','🇵🇪','🇵🇭','🇵🇳','🇵🇱','🇵🇹','🇵🇷','🇶🇦','🇷🇪','🇷🇴','🇷🇺','🇷🇼','🇼🇸','🇸🇲','🇸🇦','🇸🇳','🇷🇸','🇸🇨','🇸🇱','🇸🇬','🇸🇽','🇸🇰','🇸🇮','🇬🇸','🇸🇧','🇸🇴','🇿🇦','🇰🇷','🇸🇸','🇪🇸','🇱🇰','🇧🇱','🇸🇭','🇰🇳','🇱🇨','🇵🇲','🇻🇨','🇸🇩','🇸🇷','🇸🇿','🇸🇪','🇨🇭','🇸🇾','🇹🇼','🇹🇯','🇹🇿','🇹🇭','🇹🇱','🇹🇬','🇹🇰','🇹🇴','🇹🇹','🇹🇳','🇹🇷','🇹🇲','🇹🇨','🇹🇻','🇻🇮','🇺🇬','🇺🇦','🇦🇪','🇬🇧','🏴󠁧󠁢󠁥󠁮󠁧󠁿','🏴󠁧󠁢󠁳󠁣󠁴󠁿','🏴󠁧󠁢󠁷󠁬󠁳󠁿','🇺🇳','🇺🇸','🇺🇾','🇺🇿','🇻🇺','🇻🇦','🇻🇪','🇻🇳','🇼🇫','🇪🇭','🇾🇪','🇿🇲','🇿🇼'
    ,'🫨','🩷','🩵','🩶','🫸','🫸🏻','🫸🏼','🫸🏽','🫸🏾','🫸🏿','🫷','🫷🏻','🫷🏼','🫷🏽','🫷🏾','🫷🏿','🫏','🫎','🪿','🐦‍⬛','🪽','🪼','🪻','🫛','🫚','🪭','🪮','🪈','🪇','🪯','🛜'
    ,'🫠','🫢','🫣','🫡','🫥','🫤','🥹','🫱','🫱🏻','🫱🏼','🫱🏽','🫱🏾','🫱🏿','🫲','🫲🏻','🫲🏼','🫲🏽','🫲🏾','🫲🏿','🫳','🫳🏻','🫳🏼','🫳🏽','🫳🏾','🫳🏿','🫴','🫴🏻','🫴🏼','🫴🏽','🫴🏾','🫴🏿','🫰','🫰🏻','🫰🏼','🫰🏽','🫰🏾','🫰🏿','🫵','🫵🏻','🫵🏼','🫵🏽','🫵🏾','🫵🏿','🫶','🫶🏻','🫶🏼','🫶🏽','🫶🏾','🫶🏿','🤝🏻','🤝🏼','🤝🏽','🤝🏾','🤝🏿','🫱🏻‍🫲🏼','🫱🏻‍🫲🏽','🫱🏻‍🫲🏾','🫱🏻‍🫲🏿','🫱🏼‍🫲🏻','🫱🏼‍🫲🏽','🫱🏼‍🫲🏾','🫱🏼‍🫲🏿','🫱🏽‍🫲🏻','🫱🏽‍🫲🏼','🫱🏽‍🫲🏾','🫱🏽‍🫲🏿','🫱🏾‍🫲🏻','🫱🏾‍🫲🏼','🫱🏾‍🫲🏽','🫱🏾‍🫲🏿','🫱🏿‍🫲🏻','🫱🏿‍🫲🏼','🫱🏿‍🫲🏽','🫱🏿‍🫲🏾','🫦','🫅','🫅🏻','🫅🏼','🫅🏽','🫅🏾','🫅🏿','🫃','🫃🏻','🫃🏼','🫃🏽','🫃🏾','🫃🏿','🫄','🫄🏻','🫄🏼','🫄🏽','🫄🏾','🫄🏿','🧌','🪸','🪷','🪹','🪺','🫘','🫗','🫙','🛝','🛞','🛟','🪬','🪩','🪫','🩼','🩻','🫧','🪪','🟰'
    ,'😮‍💨','😵‍💫','😶‍🌫️','❤️‍🔥','❤️‍🩹','🧔‍♀️','🧔🏻‍♀️','🧔🏼‍♀️','🧔🏽‍♀️','🧔🏾‍♀️','🧔🏿‍♀️','🧔‍♂️','🧔🏻‍♂️','🧔🏼‍♂️','🧔🏽‍♂️','🧔🏾‍♂️','🧔🏿‍♂️','💑🏻','💑🏼','💑🏽','💑🏾','💑🏿','💏🏻','💏🏼','💏🏽','💏🏾','💏🏿','👨🏻‍❤️‍👨🏻','👨🏻‍❤️‍👨🏼','👨🏻‍❤️‍👨🏽','👨🏻‍❤️‍👨🏾','👨🏻‍❤️‍👨🏿','👨🏼‍❤️‍👨🏻','👨🏼‍❤️‍👨🏼','👨🏼‍❤️‍👨🏽','👨🏼‍❤️‍👨🏾','👨🏼‍❤️‍👨🏿','👨🏽‍❤️‍👨🏻','👨🏽‍❤️‍👨🏼','👨🏽‍❤️‍👨🏽','👨🏽‍❤️‍👨🏾','👨🏽‍❤️‍👨🏿','👨🏾‍❤️‍👨🏻','👨🏾‍❤️‍👨🏼','👨🏾‍❤️‍👨🏽','👨🏾‍❤️‍👨🏾','👨🏾‍❤️‍👨🏿','👨🏿‍❤️‍👨🏻','👨🏿‍❤️‍👨🏼','👨🏿‍❤️‍👨🏽','👨🏿‍❤️‍👨🏾','👨🏿‍❤️‍👨🏿','👩🏻‍❤️‍👨🏻','👩🏻‍❤️‍👨🏼','👩🏻‍❤️‍👨🏽','👩🏻‍❤️‍👨🏾','👩🏻‍❤️‍👨🏿','👩🏻‍❤️‍👩🏻','👩🏻‍❤️‍👩🏼','👩🏻‍❤️‍👩🏽','👩🏻‍❤️‍👩🏾','👩🏻‍❤️‍👩🏿','👩🏼‍❤️‍👨🏻','👩🏼‍❤️‍👨🏼','👩🏼‍❤️‍👨🏽','👩🏼‍❤️‍👨🏾','👩🏼‍❤️‍👨🏿','👩🏼‍❤️‍👩🏻','👩🏼‍❤️‍👩🏼','👩🏼‍❤️‍👩🏽','👩🏼‍❤️‍👩🏾','👩🏼‍❤️‍👩🏿','👩🏽‍❤️‍👨🏻','👩🏽‍❤️‍👨🏼','👩🏽‍❤️‍👨🏽','👩🏽‍❤️‍👨🏾','👩🏽‍❤️‍👨🏿','👩🏽‍❤️‍👩🏻','👩🏽‍❤️‍👩🏼','👩🏽‍❤️‍👩🏽','👩🏽‍❤️‍👩🏾','👩🏽‍❤️‍👩🏿','👩🏾‍❤️‍👨🏻','👩🏾‍❤️‍👨🏼','👩🏾‍❤️‍👨🏽','👩🏾‍❤️‍👨🏾','👩🏾‍❤️‍👨🏿','👩🏾‍❤️‍👩🏻','👩🏾‍❤️‍👩🏼','👩🏾‍❤️‍👩🏽','👩🏾‍❤️‍👩🏾','👩🏾‍❤️‍👩🏿','👩🏿‍❤️‍👨🏻','👩🏿‍❤️‍👨🏼','👩🏿‍❤️‍👨🏽','👩🏿‍❤️‍👨🏾','👩🏿‍❤️‍👨🏿','👩🏿‍❤️‍👩🏻','👩🏿‍❤️‍👩🏼','👩🏿‍❤️‍👩🏽','👩🏿‍❤️‍👩🏾','👩🏿‍❤️‍👩🏿','🧑🏻‍❤️‍🧑🏼','🧑🏻‍❤️‍🧑🏽','🧑🏻‍❤️‍🧑🏾','🧑🏻‍❤️‍🧑🏿','🧑🏼‍❤️‍🧑🏻','🧑🏼‍❤️‍🧑🏽','🧑🏼‍❤️‍🧑🏾','🧑🏼‍❤️‍🧑🏿','🧑🏽‍❤️‍🧑🏻','🧑🏽‍❤️‍🧑🏼','🧑🏽‍❤️‍🧑🏾','🧑🏽‍❤️‍🧑🏿','🧑🏾‍❤️‍🧑🏻','🧑🏾‍❤️‍🧑🏼','🧑🏾‍❤️‍🧑🏽','🧑🏾‍❤️‍🧑🏿','🧑🏿‍❤️‍🧑🏻','🧑🏿‍❤️‍🧑🏼','🧑🏿‍❤️‍🧑🏽','🧑🏿‍❤️‍🧑🏾','👨🏻‍❤️‍💋‍👨🏻','👨🏻‍❤️‍💋‍👨🏼','👨🏻‍❤️‍💋‍👨🏽','👨🏻‍❤️‍💋‍👨🏾','👨🏻‍❤️‍💋‍👨🏿','👨🏼‍❤️‍💋‍👨🏻','👨🏼‍❤️‍💋‍👨🏼','👨🏼‍❤️‍💋‍👨🏽','👨🏼‍❤️‍💋‍👨🏾','👨🏼‍❤️‍💋‍👨🏿','👨🏽‍❤️‍💋‍👨🏻','👨🏽‍❤️‍💋‍👨🏼','👨🏽‍❤️‍💋‍👨🏽','👨🏽‍❤️‍💋‍👨🏾','👨🏽‍❤️‍💋‍👨🏿','👨🏾‍❤️‍💋‍👨🏻','👨🏾‍❤️‍💋‍👨🏼','👨🏾‍❤️‍💋‍👨🏽','👨🏾‍❤️‍💋‍👨🏾','👨🏾‍❤️‍💋‍👨🏿','👨🏿‍❤️‍💋‍👨🏻','👨🏿‍❤️‍💋‍👨🏼','👨🏿‍❤️‍💋‍👨🏽','👨🏿‍❤️‍💋‍👨🏾','👨🏿‍❤️‍💋‍👨🏿','👩🏻‍❤️‍💋‍👨🏻','👩🏻‍❤️‍💋‍👨🏼','👩🏻‍❤️‍💋‍👨🏽','👩🏻‍❤️‍💋‍👨🏾','👩🏻‍❤️‍💋‍👨🏿','👩🏻‍❤️‍💋‍👩🏻','👩🏻‍❤️‍💋‍👩🏼','👩🏻‍❤️‍💋‍👩🏽','👩🏻‍❤️‍💋‍👩🏾','👩🏻‍❤️‍💋‍👩🏿','👩🏼‍❤️‍💋‍👨🏻','👩🏼‍❤️‍💋‍👨🏼','👩🏼‍❤️‍💋‍👨🏽','👩🏼‍❤️‍💋‍👨🏾','👩🏼‍❤️‍💋‍👨🏿','👩🏼‍❤️‍💋‍👩🏻','👩🏼‍❤️‍💋‍👩🏼','👩🏼‍❤️‍💋‍👩🏽','👩🏼‍❤️‍💋‍👩🏾','👩🏼‍❤️‍💋‍👩🏿','👩🏽‍❤️‍💋‍👨🏻','👩🏽‍❤️‍💋‍👨🏼','👩🏽‍❤️‍💋‍👨🏽','👩🏽‍❤️‍💋‍👨🏾','👩🏽‍❤️‍💋‍👨🏿','👩🏽‍❤️‍💋‍👩🏻','👩🏽‍❤️‍💋‍👩🏼','👩🏽‍❤️‍💋‍👩🏽','👩🏽‍❤️‍💋‍👩🏾','👩🏽‍❤️‍💋‍👩🏿','👩🏾‍❤️‍💋‍👨🏻','👩🏾‍❤️‍💋‍👨🏼','👩🏾‍❤️‍💋‍👨🏽','👩🏾‍❤️‍💋‍👨🏾','👩🏾‍❤️‍💋‍👨🏿','👩🏾‍❤️‍💋‍👩🏻','👩🏾‍❤️‍💋‍👩🏼','👩🏾‍❤️‍💋‍👩🏽','👩🏾‍❤️‍💋‍👩🏾','👩🏾‍❤️‍💋‍👩🏿','👩🏿‍❤️‍💋‍👨🏻','👩🏿‍❤️‍💋‍👨🏼','👩🏿‍❤️‍💋‍👨🏽','👩🏿‍❤️‍💋‍👨🏾','👩🏿‍❤️‍💋‍👨🏿','👩🏿‍❤️‍💋‍👩🏻','👩🏿‍❤️‍💋‍👩🏼','👩🏿‍❤️‍💋‍👩🏽','👩🏿‍❤️‍💋‍👩🏾','👩🏿‍❤️‍💋‍👩🏿','🧑🏻‍❤️‍💋‍🧑🏼','🧑🏻‍❤️‍💋‍🧑🏽','🧑🏻‍❤️‍💋‍🧑🏾','🧑🏻‍❤️‍💋‍🧑🏿','🧑🏼‍❤️‍💋‍🧑🏻','🧑🏼‍❤️‍💋‍🧑🏽','🧑🏼‍❤️‍💋‍🧑🏾','🧑🏼‍❤️‍💋‍🧑🏿','🧑🏽‍❤️‍💋‍🧑🏻','🧑🏽‍❤️‍💋‍🧑🏼','🧑🏽‍❤️‍💋‍🧑🏾','🧑🏽‍❤️‍💋‍🧑🏿','🧑🏾‍❤️‍💋‍🧑🏻','🧑🏾‍❤️‍💋‍🧑🏼','🧑🏾‍❤️‍💋‍🧑🏽','🧑🏾‍❤️‍💋‍🧑🏿','🧑🏿‍❤️‍💋‍🧑🏻','🧑🏿‍❤️‍💋‍🧑🏼','🧑🏿‍❤️‍💋‍🧑🏽','🧑🏿‍❤️‍💋‍🧑🏾'
]

def random_emoji_string(length):
    return ''.join(random.choices(emojis, k=length))

# write wildcard concerts to ical
c = Calendar()
for index, row in df[df['m_rating'] == '🤡Wildcard'].iterrows():
    e = Event()

    # add wildcard-name! Who needs text when you're gonna watch a wildcard concert!!
    e.name = "🤡: " + row['emoji_beskrivelse']
    e.location = row['location']

    e.begin = row['start_datetime']

    # assume all concerts are 1 hour
    e.end = row['start_datetime'] + timedelta(hours=1) # assume all concerts are 1 hour
    
    e.description = random_emoji_string(25)


    c.events.add(e)

# c.events
with open('wildcard.ics', 'w') as my_file:
    my_file.writelines(c.serialize_iter())
